In [9]:
from transformers import AutoTokenizer, AutoModelForPreTraining, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("classla/bcms-bertic")
#token_classification = AutoModelForTokenClassification.from_pretrained("classla/bcms-bertic")


model = AutoModelForPreTraining.from_pretrained("classla/bcms-bertic")

In [15]:
from simpletransformers.custom_models.models import ElectraForMultiLabelSequenceClassification
model = ElectraForMultiLabelSequenceClassification.from_pretrained("classla/bcms-bertic-geo")


In [17]:
ElectraForMultiLabelSequenceClassification.load_model("classla/bcms-bertic-geo")

AttributeError: type object 'ElectraForMultiLabelSequenceClassification' has no attribute 'load_model'

In [3]:
#https://github.com/clarinsi/geobert

from simpletransformers.classification import MultiLabelClassificationModel
import numpy as np
import pickle


class GlobalScaler():
	def __init__(self):
		self.means = None
		self.stddev = None

	def fit_transform(self, data):
		self.means = np.mean(data, axis=0)
		centereddata = data - self.means
		self.stddev = np.std(centereddata)
		return centereddata / self.stddev

	def transform(self, data):
		return (data - self.means) / self.stddev

	def inverse_transform(self, data):
		return (np.asarray(data) * self.stddev) + self.means

scl=pickle.load(open('bcms.scaler','rb'))

# Setting optional model configuration
model_args = {
    "regression": True,
    "do_lower_case": True,
    "eval_batch_size": 64,
	"loss_fct": "MAELoss",
}

# Create a ClassificationModel
model = MultiLabelClassificationModel(
    "electra",
    "classla/bcms-bertic-geo",
    num_labels=2,
    args=model_args,
	loss_fct="MAELoss",
	use_cuda=False
)





TypeError: __init__() got an unexpected keyword argument 'loss_fct'

In [63]:


import reverse_geocode

pred_inv= [[42.37963124, 19.2159317 ]]


pred_rev = reverse_geocode.search(pred_inv)
for c, r in zip(pred_inv, pred_rev):
	print(c,r)

[42.37963124, 19.2159317] {'country_code': 'ME', 'city': 'Mojanovići', 'country': 'Montenegro'}


In [12]:
import pandas as pd

In [13]:
df = pd.read_csv('data.csv')


In [14]:
#define which data to analyze

#by song_ID
df["toAnalyze"] = df["Song_ID"].str.find('oOIOCzZV') 
#df_old = df[df.toAnalyze == -1]
#df_old = df_old.drop(["toAnalyze"], axis=1)

df = df[df.toAnalyze >= 0]
df = df.reset_index()
df = df.drop(["index", "toAnalyze"], axis=1)


In [15]:
df_tagged = pd.DataFrame()

df_tagged_lyrics = pd.DataFrame()

In [16]:
songs = df["Song"]
lyrics = df["Lyrics"]

for index, song in enumerate(songs):
    tagged = json.loads(n.tag(song))
    df_tagged = df_tagged.append(tagged, ignore_index=True)
    
    text = lyrics[index]
    tagged_text = json.loads(n.tag(text))
    df_tagged_lyrics = df_tagged_lyrics.append(tagged_text, ignore_index=True)

NameError: name 'json' is not defined

In [17]:
raw_inputs=[l.splitlines() for l in lyrics.dropna()]

In [37]:
song_lyrics = [" ".join(l) for l in raw_inputs]

In [42]:
[[i, song] for i, song in enumerate(song_lyrics) if "mati" in song]

[[51,
  'svi dani, jutra i noći prazni su kao moj stan  imati nekog kraj sebe  to je moj san, moj davni san  pitanje kojim ko dijete  majci ja otkrih svoj jad još uvijek u srcu mi živi gori sad, boli sad  kaži mi kaži mama  zašto sam ja sama a Zlatka sa prvog kata  ima sestru i dva brata  pitanje kojim o dijete  majci ja otkrih svoj jad  još uvijek u srcu i živi  gori sad, boli sad  kaži mi kaži mama... još uvijek u srcu mi živi'],
 [58,
  'Iza vitra šta u provu tuče iza mora šta me na dno vuče vrate mene moji maestrali U moj porat, u moj porat mali U moj porat, u moj porat mali  Sidin opet kraj starog komina Napijen se domaćega vina Pa mi mater posteju namisti Lancuni su od ditinjstva isti Lancuni su od ditinjstva isti  Ne budi me mati Ne budi , ne zovi Sin tvoj na kušinu u ditinjstvo plovi U ditinjstvo plovi Ne budi me mati Kad zora dođe Sve grubo i teško neka u sne prođe Neka u sne prođe Ne budi me mati  Kukuriče kokot iza kuće Penšjunati igraju na buće Nestalo je vrimena i ura Dok 

In [61]:
song_lyrics[24]

'Prijatelji dolaze na vrijeme I vec su upaljene svijece Darovi se dijele Prava slika srece  I noc se ova Silverstarska trosi Uz poljupce i kolace I mnogo lijepih zelja A mozda se nekom place  Kad odu svi Kad ostanemo sami U nasoj sobi Sami ja i ti  Kad odu svi U praznickoj tami Bez nade i snova Ostat cemo mi  I ti ces reci Ti ces samo hladno raci Sretna Nova Sretna Nova  Dimnjacar zadnji ide niz cestu Jos se piju razna pica Jos minuta i ponoc tuce Nasoj ljubavi bez pokrica'

In [43]:
song_lyrics[58]

'Iza vitra šta u provu tuče iza mora šta me na dno vuče vrate mene moji maestrali U moj porat, u moj porat mali U moj porat, u moj porat mali  Sidin opet kraj starog komina Napijen se domaćega vina Pa mi mater posteju namisti Lancuni su od ditinjstva isti Lancuni su od ditinjstva isti  Ne budi me mati Ne budi , ne zovi Sin tvoj na kušinu u ditinjstvo plovi U ditinjstvo plovi Ne budi me mati Kad zora dođe Sve grubo i teško neka u sne prođe Neka u sne prođe Ne budi me mati  Kukuriče kokot iza kuće Penšjunati igraju na buće Nestalo je vrimena i ura Dok ja sanjan iza starih škura Dok ja sanjan iza starih škura  Ne budi me mati Ne budi ne zovi Sin tvoj na kušinu u ditinjstvo plovi U ditinjstvo plovi U ditinjstvo plovi Ne budi me mati Kad zora dođe Sve grubo i teško neka u snove prođe Neka u sne prođe Ne budi me mati.'

In [55]:
with open('test_lyrics.txt', 'w', encoding="utf-8") as f:
    f.write(song_lyrics[0])

In [56]:
with open('test_lyrics.txt', 'r', encoding="utf-8") as file:
    data = file.read().rstrip()

In [53]:
data

'Iza vitra šta u provu tuče iza mora šta me na dno vuče vrate mene moji maestrali U moj porat, u moj porat mali U moj porat, u moj porat mali  Sidin opet kraj starog komina Napijen se domaćega vina Pa mi mater posteju namisti Lancuni su od ditinjstva isti Lancuni su od ditinjstva isti  Ne budi me mati Ne budi , ne zovi Sin tvoj na kušinu u ditinjstvo plovi U ditinjstvo plovi Ne budi me mati Kad zora dođe Sve grubo i teško neka u sne prođe Neka u sne prođe Ne budi me mati  Kukuriče kokot iza kuće Penšjunati igraju na buće Nestalo je vrimena i ura Dok ja sanjan iza starih škura Dok ja sanjan iza starih škura  Ne budi me mati Ne budi ne zovi Sin tvoj na kušinu u ditinjstvo plovi U ditinjstvo plovi U ditinjstvo plovi Ne budi me mati Kad zora dođe Sve grubo i teško neka u snove prođe Neka u sne prođe Ne budi me mati.'

In [136]:
model_inputs = "Mrzim ovo sve aaaaaaa"

In [106]:
from transformers import AutoModelForSequenceClassification

In [120]:
import torch

In [137]:
inputs = tokenizer(model_inputs, return_tensors="pt")
labels = torch.tensor([[1, 1]], dtype=torch.float)  # need dtype=float for BCEWithLogitsLoss
outputs = model(**inputs, labels=labels)

In [374]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("classla/bcms-bertic-ner")

model = AutoModelForTokenClassification.from_pretrained("classla/bcms-bertic-ner")

In [377]:
nlp_token_class = pipeline('ner', model=model, tokenizer=tokenizer, grouped_entities=True)
ners = nlp_token_class(model_inputs)

In [376]:
ners

[{'entity_group': 'LOC',
  'score': 0.78599304,
  'word': 'moše pijade',
  'start': 371,
  'end': 382}]

In [180]:
p = TokenClassificationPipeline(model=model, tokenizer=tokenizer)
p("Ja sam Ana iz Zagreba")

[{'entity': 'B-PER',
  'score': 0.9925545,
  'index': 3,
  'word': 'Ana',
  'start': 7,
  'end': 10},
 {'entity': 'B-LOC',
  'score': 0.9909462,
  'index': 5,
  'word': 'Zagreba',
  'start': 14,
  'end': 21}]

In [215]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TokenClassificationPipeline

tokenizer = AutoTokenizer.from_pretrained("classla/bcms-bertic")
model = AutoModelForPreTraining.from_pretrained("classla/bcms-bertic")
token_classifier = AutoModelForTokenClassification.from_pretrained("classla/bcms-bertic")
p = TokenClassificationPipeline(model=model, tokenizer=tokenizer)
p('My name is Clara and I live in Berkeley, California.')

Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['classifier.weight', 'classifier.bias']
You should pr

IndexError: invalid index to scalar variable.

In [240]:
from simpletransformers.custom_models.models import ElectraForMultiLabelSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("classla/bcms-bertic-geo")

model = ElectraForMultiLabelSequenceClassification.from_pretrained("classla/bcms-bertic-geo")

In [244]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("classla/bcms-bertic-frenk-hate")

model = AutoModelForSequenceClassification.from_pretrained("classla/bcms-bertic-frenk-hate")

Downloading: 100%|██████████| 370/370 [00:00<00:00, 74.0kB/s]
Downloading: 100%|██████████| 321k/321k [00:00<00:00, 573kB/s]  
Downloading: 100%|██████████| 700k/700k [00:00<00:00, 896kB/s]  
Downloading: 100%|██████████| 721/721 [00:00<00:00, 120kB/s]
Downloading: 100%|██████████| 474M/474M [02:18<00:00, 3.58MB/s] 


In [248]:
from simpletransformers.classification import ClassificationModel
model_args = {
        "num_train_epochs": 12,
        "learning_rate": 1e-5,
        "train_batch_size": 74}

model = ClassificationModel(
    "bert", "classla/bcms-bertic-frenk-hate", use_cuda=False,
    args=model_args
    
)


### Output:
### array([0, 0])

100%|██████████| 1/1 [00:01<00:00,  1.74s/it]


array([0, 0], dtype=int64)

In [254]:
predictions, logit_output = model.predict(model_inputs[0:20])
predictions

100%|██████████| 3/3 [00:15<00:00,  5.27s/it]


array([1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
      dtype=int64)

In [262]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("EMBEDDIA/crosloengual-bert")

model = AutoModelForMaskedLM.from_pretrained("EMBEDDIA/crosloengual-bert")

Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [335]:
text=model_inputs[0]
print(text)


ne govorim s tobom, i ništa nije kao prije, čija to pisma čitaš? tišina, samo tišina lije.  hej, pogledaj me u oči, što čovjek može da proguta, ima li snage za još pet minuta? ili smo stigli do kraja, do kraja puta?  ako se rastanemo, onako pošteno i sami, što ćemo reći susjedima, poštaru i tvojoj mami? ako se rastanemo, neće se srušiti svijet, doći će novi stanari, u moše pijade 35  ne govorim s tobom, drugačija si, nisi ista, izgubili smo nepovratno, djetelinu s četiri lista.  daj, pogledaj me u oči što čovjek može da proguta, ima li snage za još pet minuta? ili smo stigli do kraja, do kraja puta?


In [336]:
tokenizer.tokenize(text)

['ne',
 'govorim',
 's',
 'tobo',
 '##m',
 ',',
 'i',
 'ništa',
 'nije',
 'kao',
 'prije',
 ',',
 'čija',
 'to',
 'pisma',
 'čita',
 '##š',
 '?',
 'tišin',
 '##a',
 ',',
 'samo',
 'tišin',
 '##a',
 'lije',
 '.',
 'he',
 '##j',
 ',',
 'pogleda',
 '##j',
 'me',
 'u',
 'oči',
 ',',
 'što',
 'čovjek',
 'može',
 'da',
 'prog',
 '##uta',
 ',',
 'ima',
 'li',
 'snage',
 'za',
 'još',
 'pet',
 'minuta',
 '?',
 'ili',
 'smo',
 'stigli',
 'do',
 'kraja',
 ',',
 'do',
 'kraja',
 'puta',
 '?',
 'ako',
 'se',
 'rasta',
 '##nemo',
 ',',
 'onako',
 'pošteno',
 'i',
 'sami',
 ',',
 'što',
 'ćemo',
 'reći',
 'susjedi',
 '##ma',
 ',',
 'pošta',
 '##ru',
 'i',
 'tvoj',
 '##oj',
 'mami',
 '?',
 'ako',
 'se',
 'rasta',
 '##nemo',
 ',',
 'neće',
 'se',
 'sruši',
 '##ti',
 'svijet',
 ',',
 'doći',
 'će',
 'novi',
 'stanar',
 '##i',
 ',',
 'u',
 'moš',
 '##e',
 'pij',
 '##ade',
 '35',
 'ne',
 'govorim',
 's',
 'tobo',
 '##m',
 ',',
 'druga',
 '##čija',
 'si',
 ',',
 'nisi',
 'ista',
 ',',
 'izgubili',
 'smo',

In [341]:
nlp_token_class = pipeline('fill-mask', model=model, tokenizer=tokenizer)
ners = nlp_token_class(text.replace("moše pijade 35", "[MASK]"))

In [342]:
ners

[{'score': 0.16262193024158478,
  'token': 1649,
  'token_str': 'stvari',
  'sequence': 'ne govorim s tobom, i ništa nije kao prije, čija to pisma čitaš? tišina, samo tišina lije. hej, pogledaj me u oči, što čovjek može da proguta, ima li snage za još pet minuta? ili smo stigli do kraja, do kraja puta? ako se rastanemo, onako pošteno i sami, što ćemo reći susjedima, poštaru i tvojoj mami? ako se rastanemo, neće se srušiti svijet, doći će novi stanari, u stvari ne govorim s tobom, drugačija si, nisi ista, izgubili smo nepovratno, djetelinu s četiri lista. daj, pogledaj me u oči što čovjek može da proguta, ima li snage za još pet minuta? ili smo stigli do kraja, do kraja puta?'},
 {'score': 0.1452377885580063,
  'token': 1114,
  'token_str': 'biti',
  'sequence': 'ne govorim s tobom, i ništa nije kao prije, čija to pisma čitaš? tišina, samo tišina lije. hej, pogledaj me u oči, što čovjek može da proguta, ima li snage za još pet minuta? ili smo stigli do kraja, do kraja puta? ako se rasta

In [330]:
nlp_token_class = pipeline('fill-mask', model=model, tokenizer=tokenizer)
ners = nlp_token_class("Volim pit i [MASK]")

In [331]:
ners

[{'score': 0.3062123954296112,
  'token': 11924,
  'token_str': 'jesti',
  'sequence': 'Volim pit i jesti'},
 {'score': 0.21018792688846588,
  'token': 31844,
  'token_str': 'piti',
  'sequence': 'Volim pit i piti'},
 {'score': 0.022669702768325806,
  'token': 31456,
  'token_str': 'čitati',
  'sequence': 'Volim pit i čitati'},
 {'score': 0.020960301160812378,
  'token': 9485,
  'token_str': 'uživati',
  'sequence': 'Volim pit i uživati'},
 {'score': 0.014921344816684723,
  'token': 2708,
  'token_str': 'raditi',
  'sequence': 'Volim pit i raditi'}]

In [345]:
from transformers import BertModel
model = BertModel.from_pretrained("EMBEDDIA/crosloengual-bert")


Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [347]:

outext = "Ja sam ana"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [364]:
import torch.nn as nn
from transformers import AutoModel
device = 'cpu'


In [359]:
class LinearProbeBert(nn.Module):
  def __init__(self, num_labels):
    super().__init__()
    self.bert = AutoModel.from_pretrained("EMBEDDIA/crosloengual-bert")
    self.probe = nn.Linear(self.bert.config.hidden_size, num_labels)
    self.to(device)

  def parameters(self):
    return self.probe.parameters()
  
  def forward(self, sentences):
    with torch.no_grad(): # no training of BERT parameters
      word_rep, sentence_rep = self.bert(sentences, return_dict=False)
    return self.probe(word_rep)



In [352]:
import collections

In [353]:
label_vocab = collections.defaultdict(lambda: len(label_vocab))
label_vocab['<pad>'] = 0

In [358]:
label_vocab

defaultdict(<function __main__.<lambda>()>, {'<pad>': 0})

Downloading:  15%|█▍        | 60.5M/416M [00:36<01:37, 3.82MB/s]

In [365]:
# the model should return a tensor of shape (batch size, sequence length, number of labels)
bert_model = LinearProbeBert(len(label_vocab))
y = bert_model(torch.tensor([[0, 1, 2], [3, 4, 5]]).to(device))
print(y.shape)

Some weights of the model checkpoint at EMBEDDIA/crosloengual-bert were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([2, 3, 1])


In [366]:
y

tensor([[[-0.0753],
         [ 0.3247],
         [-0.0744]],

        [[-0.2319],
         [-0.4074],
         [-0.4529]]], grad_fn=<AddBackward0>)